In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, Flatten, Dropout, Reshape, LayerNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split



In [7]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


NEURAL NET

In [ ]:
def create_attention_transformer(input_shape):
    # Create the Input Layer
    input_layer = Input(shape=input_shape)

    
    dense_layer_one = Dense(256, activation='relu')(input_layer)
    dense_layer_two = Dense(256, activation='relu')(dense_layer_one)
    dense_layer_three = Dense(256, activation='relu')(dense_layer_two)
    dense_layer_four = Dense(128, activation='relu')(dense_layer_three)

    
    dense_layer_five = Dense(64, activation='relu')(dense_layer_four)

    dense_layer_six = Dense(32, activation='relu')(dense_layer_five)
    
    last_dense_layer = Dense(16, activation='relu')(dense_layer_six)   
    
    reshaped_layer = Reshape((1, 16))(last_dense_layer)

    
    attention = MultiHeadAttention(num_heads=3, key_dim=64)(reshaped_layer, reshaped_layer)
    print("Attention Embeddings Shape:", attention.shape)

    
    normalized_attention = LayerNormalization(epsilon=1e-6)(attention)
    print("Normalized Attention Embeddings Shape:", normalized_attention.shape)
    dropout_attention = Dropout(0.1)(normalized_attention)

    
    flattened_attention = Flatten()(dropout_attention)
    print("Flattened Attention Embeddings Shape:", flattened_attention.shape)

    
    output_layer = Dense(7, activation='softmax')(flattened_attention)

    
    attention_model = Model(inputs=input_layer, outputs=output_layer)
    return attention_model

# Define input shape
attention_input_shape = (114,)  # Example input shape

# Create the model
attention_transformer_ecg = create_attention_transformer(attention_input_shape)

# Compile the model with a decreased learning rate
attention_transformer_ecg.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])

# Print the model summary
attention_transformer_ecg.summary()

Attention Embeddings Shape: (None, 1, 16)
Normalized Attention Embeddings Shape: (None, 1, 16)
Flattened Attention Embeddings Shape: (None, 16)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 dense (Dense)                  (None, 256)          262400      ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 256)          65792       ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 256)      

In [4]:
# Load the .npy files
ab_columns = np.load("ab_columns.npy", allow_pickle=True)
coh_columns = np.load("coh_columns.npy", allow_pickle=True)
other_columns = np.load("other_columns.npy", allow_pickle=True)

# Print the loaded data
print("ab_columns:", ab_columns)
print("coh_columns:", coh_columns)
print("other_columns:", other_columns)


ab_columns: [[48.84389  46.533704 25.924618 ...  2.388181  4.271034  4.093793]
 [34.108015 22.838567 20.646824 ...  2.507125  1.333279  7.415794]
 [31.084064 28.212342 30.467865 ...  6.313069  2.739807  7.6445  ]
 ...
 [20.05537  11.734445 15.719485 ...  1.180694  1.100077  0.741887]
 [26.43057  20.721589 25.486586 ...  0.976765  1.362347  1.168174]
 [14.64494  14.570681 13.731359 ...  0.442399  0.300771  0.411511]]
coh_columns: [[85.05668  84.314205 88.042424 ... 30.893891 74.038603 51.237679]
 [84.994328 57.571481 75.298051 ... 40.297206 55.547526 63.630547]
 [43.406463 33.374232 37.783015 ... 53.49483  66.581021 80.202968]
 ...
 [59.618728 56.295958 73.067333 ... 62.032961 79.282698 80.363086]
 [73.70224  56.132581 67.605944 ... 27.534399 40.861627 49.097135]
 [82.495367 53.115312 71.716848 ... 43.366516 59.321302 51.526134]]
other_columns: [[1 'F' 31.55 ... 103.0 'Mood disorder' 'Depressive disorder']
 [2 'M' 25.0 ... 115.0 'Healthy control' 'Healthy control']
 [3 'M' 26.2 ... 83.0

In [8]:
labels = other_columns[:, 6]
X_train, X_val, y_train, y_val = train_test_split(ab_columns, labels, test_size=0.2, random_state=42, stratify=labels)

# Print shapes to verify
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

X_train shape: (681, 114)
X_val shape: (171, 114)
y_train shape: (681,)
y_val shape: (171,)
